In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import os
import warnings 

# suppress all warnings
warnings.filterwarnings('ignore')


# Fetch historical data
symbol = "EURUSD=X"
start_date = "2020-01-01"
end_date = "2023-12-31"

df = yf.download(symbol, start=start_date, end=end_date)

# Strategy implementation
df['signal'] = np.where(df['Open'] > df['Close'].shift(1), 1, 0)

# Backtesting
initial_balance = 10000
balance = initial_balance
position = 0
trades = []

for i in range(1, len(df)):
    if position == 0 and df['signal'].iloc[i] == 1:
        entry_price = df['Open'].iloc[i]
        stop_loss = entry_price - 0.0020  # 20 pips
        take_profit = entry_price + 0.0045  # 45 pips
        position = 1
    
    elif position == 1:
        current_price = df['Close'].iloc[i]
        if current_price <= stop_loss or current_price >= take_profit:
            exit_price = current_price
            pnl = (exit_price - entry_price) * 10000  # Convert to pips
            balance += pnl
            trades.append({
                'entry_date': df.index[i-1],
                'exit_date': df.index[i],
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': pnl
            })
            position = 0

# Create trades DataFrame
trades_df = pd.DataFrame(trades)
trades_df.set_index('entry_date', inplace=True)

# Calculate returns
trades_df['returns'] = trades_df['pnl'] / initial_balance

# Generate performance report
report_filename = 'eur_usd_backtest_report.html'
qs.reports.html(trades_df['returns'], output=report_filename)

# Save trades DataFrame to CSV
csv_filename = 'eur_usd_trades.csv'
trades_df.to_csv(csv_filename)

print(f"Final Balance: ${balance:.2f}")
print(f"Total Return: {(balance - initial_balance) / initial_balance:.2%}")
print(f"Number of Trades: {len(trades)}")

# Show output file locations
print("\nOutput files:")
print(f"1. Performance Report: {os.path.abspath(report_filename)}")
print(f"2. Trades CSV: {os.path.abspath(csv_filename)}")

[*********************100%%**********************]  1 of 1 completed


Final Balance: $11348.10
Total Return: 13.48%
Number of Trades: 264

Output files:
1. Performance Report: c:\Users\Wolfrank\Desktop\CodeWolf\QuantTrading\Algorithmic Backtests\QuantTrading\Algorithmic Backtests\EURUSD\eur_usd_backtest_report.html
2. Trades CSV: c:\Users\Wolfrank\Desktop\CodeWolf\QuantTrading\Algorithmic Backtests\QuantTrading\Algorithmic Backtests\EURUSD\eur_usd_trades.csv
